# Artificial Neural Network

### Importing the libraries

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
tf.__version__

'2.8.2'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

directory = "/content/drive/MyDrive/Research/Deep Learning Datasets/Churn_Modelling.csv"

Mounted at /content/drive


## Part 1 - Data Preprocessing

### Importing the dataset

In [9]:
dataset = pd.read_csv(directory)
X = dataset.iloc[:, 3:-1].values #creates matrix feature => takes all columns except the first three and last column [RowNumber, CustomerID, CustomerName and exited]
y = dataset.iloc[:, -1].values #takes only the last column

In [10]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [11]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #take the index 2 and 1 hot encode

In [13]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1])], remainder='passthrough') #index for which column
X = np.array(ct.fit_transform(X))

In [15]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

### Feature Scaling

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [18]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [19]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [20]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [21]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [22]:
#loss for binary => binary crossentropy, categorical => categorical crossentropy
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [23]:
ann.fit(X_train, y_train, batch_size=32, epochs=150)

Epoch 1/150
250/250 [==============================] - 1s 2ms/step - loss: 0.6178 - accuracy: 0.6724
Epoch 2/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4745 - accuracy: 0.7979
Epoch 3/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4400 - accuracy: 0.8084
Epoch 4/150
250/250 [==============================] - 1s 2ms/step - loss: 0.4194 - accuracy: 0.8211
Epoch 5/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4042 - accuracy: 0.8273
Epoch 6/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3898 - accuracy: 0.8361
Epoch 7/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3761 - accuracy: 0.8454
Epoch 8/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3677 - accuracy: 0.8500
Epoch 9/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3617 - accuracy: 0.8534
Epoch 10/150
250/250 [==============================] - 0s 2ms/step - loss: 0.3578 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [26]:
res = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(res > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [27]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print("Confusion Matrix\n ", cm)

acc = accuracy_score(y_test,y_pred)
print("Accuracy: ", acc)

Confusion Matrix
  [[1518   77]
 [ 192  213]]
Accuracy:  0.8655
